The data for this project is ‘mobile.csv,’ and the goal is to predict the device's price range given the mobile device characteristics. You will implement different classifiers, including Logistic Regression, Support Vector Machines, K-Nearest neighbors, and Random Forest.
Create one single train/test split with a test size of 25% of the total number of observations and a random_state=123. You will decide if you need to properly scale the data, fit each classifier on the train set, and measure the test's overall accuracy. For each classifier that requires method-specific hyper-parameters, you will determine the best choice from the following range
.2) ’
Support Vector Machine - polynomial kernel. Test degrees 1,2 and 3 and a range for C between 1 and 75 with increments
3)  
4) .5
Support Vector Machine - radially basis kernel. Test gamma between 0.0005 and 0.01 in 0.0001 increments and a range for C between 5 and 75 with increm
4)  
5)  0.5
K-Nearest neighbors - test the performance for any number of neighbors between 3 and 125. The weights could be either ‘uniform’ or ‘distance.’ 
After determining the best choice of hyperparameters for each method, show confusion matrices with the best reaccuracy.

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix 
import numpy as np 
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [18]:
data = pd.read_csv('data/mobile.csv') 

In [19]:
y = data['price_range']
x = data.drop(columns='price_range')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=123)

In [4]:
model = LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(X_train, y_train) 
print(accuracy_score(y_test,model.predict(X_test)))
pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)))

0.782


/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,0,1,2,3
0,109,10,0,0
1,15,89,23,1
2,0,16,90,31
3,0,1,12,103


In [5]:
from sklearn import svm

In [6]:
MIN_IND = 1
MAX_IND = 75
STEP_SIZE = .5   
COOLING = 0.98   

In [9]:
C_values = np.arange(1, 75.5, 0.5)  
degrees = [1, 2, 3]
def cost_function(model):
    pipe = Pipeline([['Scale', StandardScaler()], ['Model', model]])
    k_fold = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=1, scoring='neg_mean_squared_error')
    return -k_fold.mean()

def random_start():
    i = np.random.randint(MIN_IND, MAX_IND)
    j = np.random.choice([0, 1, 2])  # Degree indices for 1, 2, or 3
    pos = [i, j]
    return pos
def random_neighbour(pos, T):
    i = pos[0]
    j = pos[1]
    raw_i = np.random.normal() * STEP_SIZE * T
    delta_i = np.floor(raw_i).astype(int)
    delta_j = np.random.choice([-1, 1]) if np.random.random() > 0.5 else 0
    if delta_i == 0 and delta_j == 0:
        delta_i = np.sign(raw_i).astype(int)
        delta_j = np.random.choice([-1, 1])       
    new_i = min(MAX_IND - 1, max(MIN_IND, i + delta_i))
    new_j = min(2, max(0, j + delta_j))
    return [new_i, new_j]
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        return np.exp(-(new_cost - cost) / temperature)
def annealing(pos0=random_start()):
    i = pos0[0]
    j = pos0[1]
    C = C_values[i]
    degree = degrees[j]
    model = SVC(C=C, kernel='poly', degree=degree, max_iter=10000)
    cost = cost_function(model)
    C_states, degree_states, costs = [C], [degree], [cost]
    T = 1
    pos = pos0
    
    while T > 1e-10:
        new_pos = random_neighbour(pos, T)
        i, j = new_pos
        C = C_values[i]
        degree = degrees[j]
        model = SVC(C=C, kernel='poly', degree=degree, max_iter=10000)
        new_cost = cost_function(model)  
        if acceptance_probability(cost, new_cost, T) > np.random.random():
            cost = new_cost
            pos = new_pos
            C_states.append(C)
            degree_states.append(degree)
            costs.append(cost)
        
        T *= COOLING
    return model, np.column_stack([C_states, degree_states]), costs

In [11]:
def locate_min(a):
    smallest = min(a)
    return smallest, [index for index, element in enumerate(a) if smallest == element]

In [10]:
model,states,costs = annealing()

lowest_cost, best_index = locate_min(costs)
print('Lowest cost function value: ' + str(lowest_cost))
print('C Values')
print([states[idx,0] for idx in best_index])

print('Degree')
print([states[idx,1] for idx in best_index])

/opt/conda/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Con

NameError: name 'locate_min' is not defined

In [12]:
lowest_cost, best_index = locate_min(costs)
print('Lowest cost function value: ' + str(lowest_cost))
print('C Values')
print([states[idx,0] for idx in best_index])

print('Degree')
print([states[idx,1] for idx in best_index])

Lowest cost function value: 0.24066666666666667
C Values
[1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.

In [20]:
model = SVC(C=1.5, kernel='poly', degree=3, max_iter=10000)
X_train=StandardScaler().fit_transform(X_train)
X_test=StandardScaler().fit_transform(X_test)
model.fit(X_train, y_train) 
print(accuracy_score(y_test,model.predict(X_test)))
pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)))

0.788


,0,1,2,3
0,102,17,0,0
1,21,89,18,0
2,0,18,105,14
3,0,0,18,98


In [21]:
MIN_IND = 1
MAX_IND_C = 151  
MAX_IND_GAMMA = 50 
STEP_SIZE = .5
COOLING = 0.98

C_values = np.arange(5, 75.5, 0.5)  
gamma_values = np.arange(0.0005, 0.0101, 0.0001)  

def cost_function(model):
    pipe = Pipeline([['Scale', StandardScaler()], ['Model', model]])
    k_fold = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=1, scoring='neg_mean_squared_error')
    return -k_fold.mean()

def random_start():
    i = np.random.randint(MIN_IND, MAX_IND_C)
    j = np.random.randint(0, MAX_IND_GAMMA)  
    pos = [i, j]
    return pos

def random_neighbour(pos, T):
    i = pos[0]
    j = pos[1]
    raw_i = np.random.normal() * STEP_SIZE * T
    delta_i = np.floor(raw_i).astype(int)
    delta_j = np.random.choice([-1, 1]) if np.random.random() > 0.5 else 0
    
    # Ensure at least one of the deltas is non-zero
    if delta_i == 0 and delta_j == 0:
        delta_i = np.sign(raw_i).astype(int)
        delta_j = np.random.choice([-1, 1])
        
    new_i = min(MAX_IND_C - 1, max(MIN_IND, i + delta_i))
    new_j = min(MAX_IND_GAMMA - 1, max(0, j + delta_j))
    return [new_i, new_j]

def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        return np.exp(-(new_cost - cost) / temperature)

def annealing(pos0=random_start()):
    i = pos0[0]
    j = pos0[1]
    C = C_values[i]
    gamma = gamma_values[j]
    
    model = SVC(C=C, kernel='rbf', gamma=gamma, max_iter=10000)
    cost = cost_function(model)
    
    # Tracking lists for results
    C_states, gamma_states, costs = [C], [gamma], [cost]
    T = 1
    pos = pos0
    
    while T > 1e-10:
        new_pos = random_neighbour(pos, T)
        i, j = new_pos
        C = C_values[i]
        gamma = gamma_values[j]
        
        model = SVC(C=C, kernel='rbf', gamma=gamma, max_iter=10000)
        new_cost = cost_function(model)
        
        if acceptance_probability(cost, new_cost, T) > np.random.random():
            cost = new_cost
            pos = new_pos
            C_states.append(C)
            gamma_states.append(gamma)
            costs.append(cost)
        
        T *= COOLING
    return model, np.column_stack([C_states, gamma_states]), costs

In [22]:
model,states,costs = annealing()

lowest_cost, best_index = locate_min(costs)
print('Lowest cost function value: ' + str(lowest_cost))
print('C')
print([states[idx,0] for idx in best_index])

print('Gamma')
print([states[idx,1] for idx in best_index])

Lowest cost function value: 0.06799999999999999
C
[35.0, 32.5, 29.5]
Gamma
[0.0012000000000000003, 0.0013000000000000004, 0.0009000000000000002]


In [28]:
states[0][0]

35.0

In [31]:
model = SVC(C=states[0,0], kernel='rbf', gamma=0.0012000000000000003, max_iter=10000)
model.fit(X_train, y_train) 
print(accuracy_score(y_test,model.predict(X_test)))
pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)))

0.942


,0,1,2,3
0,116,3,0,0
1,10,115,3,0
2,0,5,127,5
3,0,0,3,113


In [46]:
from sklearn.neighbors import KNeighborsClassifier
MIN_IND = 3
MAX_IND = 125
weights_options = ['uniform', 'distance']
STEP_SIZE = 5
COOLING = 0.9

def cost_function(model):
    pipe = Pipeline([['Model', model]])
    k_fold = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=1, scoring='neg_mean_squared_error')
    return -k_fold.mean()

def random_start():
    n_neighbors = np.random.randint(MIN_IND, MAX_IND)
    weights = np.random.choice(weights_options)
    return [n_neighbors, weights]

def random_neighbour(pos, T):
    n_neighbors = pos[0]
    weights = pos[1]
    delta_n = int(np.floor(np.random.normal() * STEP_SIZE * T))
    if delta_n == 0:
        delta_n = np.sign(np.random.normal())
    new_n_neighbors = min(MAX_IND, max(MIN_IND, n_neighbors + delta_n))
    new_weights = np.random.choice(weights_options)
    return [new_n_neighbors, new_weights]

def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        p = np.exp(-(new_cost - cost) / temperature)
        return p

def annealing(pos0=random_start()):
    n_neighbors = pos0[0]
    weights = pos0[1]
    model = KNeighborsClassifier(n_neighbors=int(n_neighbors), weights=weights)
    cost = cost_function(model)
    
    states_neighbors, states_weights, costs = [n_neighbors], [weights], [cost]
    T = 1
    pos = pos0

    while T > 1e-10:
        new_pos = random_neighbour(pos, T)
        n_neighbors, weights = new_pos[0], new_pos[1]
        model = KNeighborsRegressor(n_neighbors=int(n_neighbors), weights=weights)
        new_cost = cost_function(model)
        
        if acceptance_probability(cost, new_cost, T) > np.random.random():
            cost = new_cost
            pos = new_pos
            states_neighbors.append(n_neighbors)
            states_weights.append(weights)
            costs.append(cost)
        
        T *= COOLING

    return model, np.column_stack([states_neighbors, states_weights]), costs

In [47]:
model,states,costs = annealing()

In [48]:
lowest_cost, best_index = locate_min(costs)
print('Lowest cost function value: ' + str(lowest_cost))
print('neighbors')
print([states[idx,0] for idx in best_index])

print('weights')
print([states[idx,1] for idx in best_index])

Lowest cost function value: 0.35333333333333333
neighbors
['122.0']
weights
['distance']


In [49]:
model = KNeighborsClassifier(n_neighbors=int(23), weights='distance')
model.fit(X_train, y_train) 
print(accuracy_score(y_test,model.predict(X_test)))
pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)))

0.616


,0,1,2,3
0,93,22,4,0
1,29,68,30,1
2,1,30,68,38
3,0,7,30,79


2. Repeat all the steps from 1) with complete 5-Fold cross-validations, report the choice of the best hyperparameters found, and rank the different methods according to the overall cross-validated accuracy.

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=10000)
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
model.fit(X_train, y_train)
print(accuracy_score(y_test, model.predict(X_test)))
pd.DataFrame(confusion_matrix(y_test, model.predict(X_test)))

In [ ]:
C_values = np.arange(1, 75.5, 0.5) 
degrees = [1, 2, 3]
def cost_function(model):
    pipe = Pipeline([['Scale', StandardScaler()], ['Model', model]])
    k_fold = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=5, scoring='neg_mean_squared_error')
    return -k_fold.mean()

def random_start():
    i = np.random.randint(MIN_IND, MAX_IND)
    j = np.random.choice([0, 1, 2]) 
    pos = [i, j]
    return pos
def random_neighbour(pos, T):
    i = pos[0]
    j = pos[1]
    raw_i = np.random.normal() * STEP_SIZE * T
    delta_i = np.floor(raw_i).astype(int)
    delta_j = np.random.choice([-1, 1]) if np.random.random() > 0.5 else 0
    if delta_i == 0 and delta_j == 0:
        delta_i = np.sign(raw_i).astype(int)
        delta_j = np.random.choice([-1, 1])
        
    new_i = min(MAX_IND - 1, max(MIN_IND, i + delta_i))
    new_j = min(2, max(0, j + delta_j))
    return [new_i, new_j]
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        return np.exp(-(new_cost - cost) / temperature)
def annealing(pos0=random_start()):
    i = pos0[0]
    j = pos0[1]
    C = C_values[i]
    degree = degrees[j]
    
    model = SVC(C=C, kernel='poly', degree=degree, max_iter=10000)
    cost = cost_function(model)
    C_states, degree_states, costs = [C], [degree], [cost]
    T = 1
    pos = pos0
    
    while T > 1e-10:
        new_pos = random_neighbour(pos, T)
        i, j = new_pos
        C = C_values[i]
        degree = degrees[j]
        
        model = SVC(C=C, kernel='poly', degree=degree, max_iter=10000)
        new_cost = cost_function(model)
        
        if acceptance_probability(cost, new_cost, T) > np.random.random():
            cost = new_cost
            pos = new_pos
            C_states.append(C)
            degree_states.append(degree)
            costs.append(cost)
        
        T *= COOLING
    return model, np.column_stack([C_states, degree_states]), costs

In [ ]:
lowest_cost, best_index = locate_min(costs)
print('Lowest cost function value: ' + str(lowest_cost))
print('C Values')
print([states[idx,0] for idx in best_index])

print('Degree')
print([states[idx,1] for idx in best_index])

In [ ]:
model = SVC(C=states[idx,0], kernel='poly', degree=states[idx,1], max_iter=10000)
X_train=StandardScaler().fit_transform(X_train)
X_test=StandardScaler().fit_transform(X_test)
model.fit(X_train, y_train) 
print(accuracy_score(y_test,model.predict(X_test)))
pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)))

In [52]:
MIN_IND = 1
MAX_IND_C = 141 
MAX_IND_GAMMA = 50  
STEP_SIZE = .5
COOLING = 0.98

C_values = np.arange(5, 75.5, 0.5)  
gamma_values = np.arange(0.0005, 0.0101, 0.0001) 

def cost_function(model):
    pipe = Pipeline([['Scale', StandardScaler()], ['Model', model]])
    k_fold = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=5, scoring='neg_mean_squared_error')
    return -k_fold.mean()
def random_start():
    i = np.random.randint(MIN_IND, MAX_IND_C)
    j = np.random.randint(0, MAX_IND_GAMMA)
    pos = [i, j]
    return pos

def random_neighbour(pos, T):
    i = pos[0]
    j = pos[1]
    raw_i = np.random.normal() * STEP_SIZE * T
    delta_i = np.floor(raw_i).astype(int)
    delta_j = np.random.choice([-1, 1]) if np.random.random() > 0.5 else 0
    if delta_i == 0 and delta_j == 0:
        delta_i = np.sign(raw_i).astype(int)
        delta_j = np.random.choice([-1, 1])
        
    new_i = min(MAX_IND_C - 1, max(MIN_IND, i + delta_i))
    new_j = min(MAX_IND_GAMMA - 1, max(0, j + delta_j))
    return [new_i, new_j]

def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        return np.exp(-(new_cost - cost) / temperature)
def annealing(pos0=random_start()):
    i = pos0[0]
    j = pos0[1]
    C = C_values[i]
    gamma = gamma_values[j]
    
    model = SVC(C=C, kernel='rbf', gamma=gamma, max_iter=10000)
    cost = cost_function(model)
    
    C_states, gamma_states, costs = [C], [gamma], [cost]
    T = 1
    pos = pos0
    
    while T > 1e-10:
        new_pos = random_neighbour(pos, T)
        i, j = new_pos
        C = C_values[i]
        gamma = gamma_values[j]
        
        model = SVC(C=C, kernel='rbf', gamma=gamma, max_iter=10000)
        new_cost = cost_function(model)
        
        if acceptance_probability(cost, new_cost, T) > np.random.random():
            cost = new_cost
            pos = new_pos
            C_states.append(C)
            gamma_states.append(gamma)
            costs.append(cost)
        
        T *= COOLING
    return model, np.column_stack([C_states, gamma_states]), costs

In [53]:
model,states,costs = annealing()

lowest_cost, best_index = locate_min(costs)
print('Lowest cost function value: ' + str(lowest_cost))
print('C')
print([states[idx,0] for idx in best_index])

print('Gamma')
print([states[idx,1] for idx in best_index])

Lowest cost function value: 0.07133333333333333
C
[40.5]
Gamma
[0.0013000000000000004]


In [55]:
model = SVC(C=40.5, kernel='rbf', gamma=0.0013000000000000004, max_iter=10000)
model.fit(X_train, y_train) 
print(accuracy_score(y_test,model.predict(X_test)))
pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)))

0.946


,0,1,2,3
0,116,3,0,0
1,10,116,2,0
2,0,4,128,5
3,0,0,3,113


In [56]:
from sklearn.neighbors import KNeighborsClassifier
MIN_IND = 3
MAX_IND = 125
weights_options = ['uniform', 'distance']
STEP_SIZE = 5
COOLING = 0.9

def cost_function(model):
    pipe = Pipeline([['Model', model]])
    k_fold = cross_val_score(estimator=pipe, X=X_train, y=y_train, cv=5, scoring='neg_mean_squared_error')
    return -k_fold.mean()
def random_start():
    n_neighbors = np.random.randint(MIN_IND, MAX_IND)
    weights = np.random.choice(weights_options)
    return [n_neighbors, weights]
def random_neighbour(pos, T):
    n_neighbors = pos[0]
    weights = pos[1]
    delta_n = int(np.floor(np.random.normal() * STEP_SIZE * T))
    if delta_n == 0:
        delta_n = np.sign(np.random.normal())
    new_n_neighbors = min(MAX_IND, max(MIN_IND, n_neighbors + delta_n))
    new_weights = np.random.choice(weights_options)
    return [new_n_neighbors, new_weights]
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        p = np.exp(-(new_cost - cost) / temperature)
        return p
def annealing(pos0=random_start()):
    n_neighbors = pos0[0]
    weights = pos0[1]
    model = KNeighborsClassifier(n_neighbors=int(n_neighbors), weights=weights)
    cost = cost_function(model)
    
    states_neighbors, states_weights, costs = [n_neighbors], [weights], [cost]
    T = 1
    pos = pos0

    while T > 1e-10:
        new_pos = random_neighbour(pos, T)
        n_neighbors, weights = new_pos[0], new_pos[1]
        model = KNeighborsRegressor(n_neighbors=int(n_neighbors), weights=weights)
        new_cost = cost_function(model)
        
        if acceptance_probability(cost, new_cost, T) > np.random.random():
            cost = new_cost
            pos = new_pos
            states_neighbors.append(n_neighbors)
            states_weights.append(weights)
            costs.append(cost)
        
        T *= COOLING

    return model, np.column_stack([states_neighbors, states_weights]), costs

In [57]:
model,states,costs = annealing()

In [58]:
lowest_cost, best_index = locate_min(costs)
print('Lowest cost function value: ' + str(lowest_cost))
print('neighbors')
print([states[idx,0] for idx in best_index])

print('weights')
print([states[idx,1] for idx in best_index])

Lowest cost function value: 0.459136402147449
neighbors
['10.0', '10.0']
weights
['distance', 'distance']


In [61]:
model = KNeighborsClassifier(n_neighbors=int(10), weights='distance')
model.fit(X_train, y_train) 
print(accuracy_score(y_test,model.predict(X_test)))
pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)))

0.566


,0,1,2,3
0,82,31,6,0
1,31,57,35,5
2,5,29,62,41
3,0,7,27,82
